In [2]:
import pandas as pd
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from sklearn.metrics import mean_squared_error, r2_score, balanced_accuracy_score, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import random
from collections import defaultdict
import pickle
import seaborn as sns
from CGRtools import RDFRead
from CGRtools.files import RDFWrite
from sklearn.utils import resample

In [161]:
def descriptors(file):
    data = pd.read_csv(f'datasets/{file}', names=['smiles', 'chembl_id', 'pKi'])     
    data['activity'] = np.where(data.pKi >= 7, 1, 0)
    mols = [Chem.MolFromSmiles(str(m)) for m in data['smiles']]
    fps = [AllChem.GetMorganFingerprintAsBitVect(m,2,nBits=1024) for m in mols]
    np_fps = []

    for fp in fps:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fp,arr)
        np_fps.append(arr)
    np_fps = pd.DataFrame(np_fps)
    
    x_train_external, x_test_external, y_r_train_external, y_r_test_external = train_test_split(np_fps, pd.DataFrame(data['pKi']), test_size=0.2, random_state=42)
    x_train_external, x_test_external, y_c_train_external, y_c_test_external = train_test_split(np_fps, pd.DataFrame(data['activity']), test_size=0.2, random_state=42)
    
    return x_train_external, x_test_external, y_r_train_external, y_r_test_external, y_c_train_external, y_c_test_external

In [26]:
def model(x_train_external, x_test_external, y_r_train_external, y_r_test_external, y_c_train_external, y_c_test_external):
    
    rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
    rfr = RandomForestRegressor(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
    
    rfc.fit(x_train_external, y_c_train_external)
    rfr.fit(x_train_external, y_r_train_external)
    
    y_c_pred = rfc.predict(x_test_external)
    ba = balanced_accuracy_score(y_c_test_external, y_c_pred)
    proba = rfc.predict_proba(x_test_external)
    fpr, tpr, _ = roc_curve(y_c_test_external, [x[1] for x in proba])
    AUC = auc(fpr, tpr)
    
    y_r_pred = rfr.predict(x_test_external)
    r2 = r2_score(y_r_test_external, y_r_pred)
    rmse = mean_squared_error(y_r_test_external, y_r_pred, squared=False)

    return ba, AUC, r2, rmse

In [27]:
results = dict()
for element in tqdm(os.listdir('/home/ilnura/active learning/datasets')):
    x_train_external, x_test_external, y_r_train_external, y_r_test_external, y_c_train_external, y_c_test_external = descriptors(element)
    results[element] = model(x_train_external, x_test_external, y_r_train_external, y_r_test_external, y_c_train_external, y_c_test_external)
with open ('results/result_on_all_dataset.pickle', 'wb') as f:
    pickle.dump(results, f)

  0%|          | 1/200 [00:05<18:10,  5.48s/it]

{'CHEMBL219.smi': (0.778749186727391, 0.866745283018868, 0.5502368943703845, 0.6916123666097533)}


  1%|          | 2/200 [00:10<17:12,  5.21s/it]

{'CHEMBL219.smi': (0.778749186727391, 0.866745283018868, 0.5502368943703845, 0.6916123666097533), 'CHEMBL3769.smi': (0.8128205128205128, 0.8759368836291913, 0.6889971500710499, 0.7424192223080028)}


  2%|▏         | 3/200 [00:14<16:37,  5.06s/it]

...

{'CHEMBL219.smi': (0.778749186727391, 0.866745283018868, 0.5502368943703845, 0.6916123666097533), 'CHEMBL3769.smi': (0.8128205128205128, 0.8759368836291913, 0.6889971500710499, 0.7424192223080028), 'CHEMBL4308.smi': (0.6359036144578313, 0.8024096385542169, 0.43849409505928216, 0.9062924391316061), 'CHEMBL240.smi': (0.6539926851569643, 0.8792288936299909, 0.5638949741755404, 0.6430602302686266), 'CHEMBL2326.smi': (0.8740031897926634, 0.9303561935140882, 0.6502961989861795, 0.7253961826392237), 'CHEMBL3798.smi': (0.8020114942528735, 0.9379310344827585, 0.7235068808453671, 0.8376882752529914), 'CHEMBL3223.smi': (0.7784730913642053, 0.8397997496871089, 0.3739599304596346, 0.7896441807118654), 'CHEMBL2094121.smi': (0.7043650793650793, 0.8465608465608465, 0.5493960247578289, 0.7042469083205187), 'CHEMBL255.smi': (0.836996336996337, 0.9179639804639804, 0.7224824643361817, 0.5298230441662558), 'CHEMBL225.smi': (0.7591099032757664, 0.8485579030419916, 0.5573754615038095, 0.7068130931414226), 'C

100%|█████████▉| 199/200 [16:38<00:04,  4.80s/it]

{'CHEMBL219.smi': (0.778749186727391, 0.866745283018868, 0.5502368943703845, 0.6916123666097533), 'CHEMBL3769.smi': (0.8128205128205128, 0.8759368836291913, 0.6889971500710499, 0.7424192223080028), 'CHEMBL4308.smi': (0.6359036144578313, 0.8024096385542169, 0.43849409505928216, 0.9062924391316061), 'CHEMBL240.smi': (0.6539926851569643, 0.8792288936299909, 0.5638949741755404, 0.6430602302686266), 'CHEMBL2326.smi': (0.8740031897926634, 0.9303561935140882, 0.6502961989861795, 0.7253961826392237), 'CHEMBL3798.smi': (0.8020114942528735, 0.9379310344827585, 0.7235068808453671, 0.8376882752529914), 'CHEMBL3223.smi': (0.7784730913642053, 0.8397997496871089, 0.3739599304596346, 0.7896441807118654), 'CHEMBL2094121.smi': (0.7043650793650793, 0.8465608465608465, 0.5493960247578289, 0.7042469083205187), 'CHEMBL255.smi': (0.836996336996337, 0.9179639804639804, 0.7224824643361817, 0.5298230441662558), 'CHEMBL225.smi': (0.7591099032757664, 0.8485579030419916, 0.5573754615038095, 0.7068130931414226), 'C

100%|██████████| 200/200 [16:43<00:00,  5.02s/it]

{'CHEMBL219.smi': (0.778749186727391, 0.866745283018868, 0.5502368943703845, 0.6916123666097533), 'CHEMBL3769.smi': (0.8128205128205128, 0.8759368836291913, 0.6889971500710499, 0.7424192223080028), 'CHEMBL4308.smi': (0.6359036144578313, 0.8024096385542169, 0.43849409505928216, 0.9062924391316061), 'CHEMBL240.smi': (0.6539926851569643, 0.8792288936299909, 0.5638949741755404, 0.6430602302686266), 'CHEMBL2326.smi': (0.8740031897926634, 0.9303561935140882, 0.6502961989861795, 0.7253961826392237), 'CHEMBL3798.smi': (0.8020114942528735, 0.9379310344827585, 0.7235068808453671, 0.8376882752529914), 'CHEMBL3223.smi': (0.7784730913642053, 0.8397997496871089, 0.3739599304596346, 0.7896441807118654), 'CHEMBL2094121.smi': (0.7043650793650793, 0.8465608465608465, 0.5493960247578289, 0.7042469083205187), 'CHEMBL255.smi': (0.836996336996337, 0.9179639804639804, 0.7224824643361817, 0.5298230441662558), 'CHEMBL225.smi': (0.7591099032757664, 0.8485579030419916, 0.5573754615038095, 0.7068130931414226), 'C

In [140]:
df = pd.DataFrame(results).T
df.columns = ['ba', 'auc', 'r2', 'rmse']
best_ba = list(df.sort_values(by=['ba'], ascending=False).index)[:10]
worst_ba = list(df.sort_values(by=['ba'], ascending=True).index)[:10]
best_r2 = list(df.sort_values(by=['r2'], ascending=False).index)[:10]
worst_r2 = list(df.sort_values(by=['r2'], ascending=True).index)[:10]
selected_datasets = {'best_ba': best_ba, 'worst_ba': worst_ba, 'best_r2': best_r2, 'worst_r2': worst_r2}
with open ('results/selected_datasets.pickle', 'wb') as f:
    pickle.dump(selected_datasets, f)

In [158]:
datasets = set()
for element in selected_datasets.values():
    for dataset in element:
        datasets.add(dataset)

In [162]:
for file in datasets:
    x_and_ys = descriptors(file)    
    with open(f'x_and_y/{file}.pickle', "wb") as f:
        pickle.dump(len(x_and_ys), f)
        for value in x_and_ys:
            pickle.dump(value, f)

In [4]:
def active_learning_classification(dataset, method):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[4], data[5]
    x_train_in, x_test_in, y_train_in, y_test_in, = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    results = []
    iteration = 0
    while y_train_in.shape[0] <= y_train_ex.shape[0]:
        
        rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfc.fit(x_train_in.values, y_train_in['activity'].values)
        classes = set(y_train_in['activity'].values)
        
        pred = rfc.predict(x_test_ex.values)
        proba = rfc.predict_proba(x_test_ex.values)
        if len(classes) == 1:
            if 1 in classes:
                proba = np.insert(proba, 0, 0, axis = 1)
            else:
                proba = np.insert(proba, 1, 0, axis = 1)
        ba = balanced_accuracy_score(y_test_ex.values, pred)
        fpr, tpr, _ = roc_curve(y_test_ex.values, [x[1] for x in proba])
        AUC = auc(fpr, tpr)
        results.append([iteration, ba, AUC])
       
        if y_test_in.shape[0] != 0:
        
            probs = rfc.predict_proba(x_test_in.values)
            probs_difference, prob_of_label_1, prob_bernulli = [], [], []
            for n, prob in enumerate(probs):
                try:
                    prob_bernulli.append([x_test_in.index[n], prob[1]*(1-prob[1])])
                    probs_difference.append([x_test_in.index[n], abs(prob[0]-prob[1])])
                    prob_of_label_1.append([x_test_in.index[n], prob[1]])
                except IndexError:
                    probs_difference.append([x_test_in.index[n], 1])
                    prob_bernulli.append([x_test_in.index[n], 0])
                    if 1 in classes:
                        prob_of_label_1.append([x_test_in.index[n], 1]) 
                    else:
                        prob_of_label_1.append([x_test_in.index[n], 0])

            least_sure = [x[0] for x in sorted(probs_difference, key=lambda x: x[1], reverse=False)][:5]
            most_sure = [x[0] for x in sorted(prob_of_label_1, key=lambda x: x[1], reverse=True)][:5]
            least_sure_bernulli = [x[0] for x in sorted(prob_bernulli, key=lambda x: x[1], reverse=True)][:5]
            

            if method == 'exploration':
                adding_points = least_sure
            elif method == 'exploitation':
                adding_points = most_sure
            elif method == 'bernulli':
                adding_points = least_sure_bernulli
            else:
                try:
                    adding_points = random.sample(list(x_test_in.index), 5)
                except ValueError:
                    adding_points = list(x_test_in.index)

            for point in adding_points:
                x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
                x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
        else:
            break

        iteration += 1 

    return results

In [243]:
#result_on_all_dataset_selected_data = dict()
for dataset in tqdm(selected_datasets['worst_ba']):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[4], data[5]
    rfc = RandomForestClassifier(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
    rfc.fit(x_train_ex.values, y_train_ex['activity'].values)
    pred = rfc.predict(x_test_ex.values)
    proba = rfc.predict_proba(x_test_ex.values)
    ba = balanced_accuracy_score(y_test_ex.values, pred)
    fpr, tpr, _ = roc_curve(y_test_ex.values, [x[1] for x in proba])
    AUC = auc(fpr, tpr)
    result_on_all_dataset_selected_data[dataset] = [ba, AUC]

100%|██████████| 10/10 [00:19<00:00,  1.99s/it]


In [5]:
def classification(best_or_worst_datasets):
    result = defaultdict(dict)
    for dataset in tqdm(selected_datasets[f'{best_or_worst_datasets}_ba']):
        for i in tqdm(range(10)): 
            result_for_iteration = dict()
            for method in tqdm(['exploration', 'exploitation', 'random','bernulli']):  
                result_for_iteration[method] = active_learning_classification(dataset,method)
            result[dataset][i] = result_for_iteration
        with open(f'new_results/results_bernulli_for_{best_or_worst_datasets}_ba.pickle', 'wb') as f:
            pickle.dump(result, f)
    return result

In [ ]:
def active_learning_regression(dataset, method):
    data = []
    with open(f'x_and_y/{dataset}.pickle', "rb") as f:
        for _ in range(pickle.load(f)):
            data.append(pickle.load(f))
    x_train_ex, x_test_ex, y_train_ex, y_test_ex = data[0], data[1], data[2], data[3]
    x_train_in, x_test_in, y_train_in, y_test_in = train_test_split(x_train_ex, y_train_ex, test_size=len(y_train_ex)-10)
    results = []
    iteration = 0
    while y_train_in.shape[0] <= y_train_ex.shape[0]:
        
        rfr = RandomForestRegressor(random_state=42, n_estimators=500, max_features='log2', n_jobs=20)
        rfr.fit(x_train_in.values, y_train_in['pKi'].values)
        
        pred_ex = rfr.predict(x_test_ex.values)        
        r2 = r2_score(y_test_ex.values, pred_ex)
        rmse = mean_squared_error(y_test_ex.values, pred_ex, squared=False)
        _ , _ , AUC = REC(y_test_ex.values, pred_ex)
        results.append([iteration, r2, rmse, AUC])
        
        if y_test_in.shape[0] != 0:
            if method == 'exploitation':
                pred_in = rfr.predict(x_test_in.values)      
                pred_values = []
                for n, value in enumerate(pred_in):
                    pred_values.append([x_test_in.index[n], value])
                adding_points = [x[0] for x in sorted(pred_values, key=lambda x: x[1], reverse=True)][:5]
            
​
            elif method == 'exploration':
                results_expl = dict()
                for n in range(7):
                    x_bootsrap, y_bootstrap = resample(x_train_in, y_train_in, replace=True)
                    results_expl[n] = rfr.predict(x_test_in.values)
                df = pd.DataFrame(results_expl)
                df.index = x_test_in.index
                df = df.T
                adding_points = list(df.var().nlargest(5).index)
            else:
                try:
                    adding_points = random.sample(list(x_test_in.index), 5)
                except ValueError:
                    adding_points = list(x_test_in.index)
​
            for point in adding_points:
                x_train_in, y_train_in = x_train_in.append(x_test_in.loc[point]), y_train_in.append(y_test_in.loc[point])
                x_test_in, y_test_in = x_test_in.drop(point), y_test_in.drop(point)
        else:
            break
​
        iteration += 1 
​
    return results

In [ ]:
def regression(best_or_worst_datasets):
    result = defaultdict(dict)
    for dataset in tqdm(selected_datasets[f'{best_or_worst_datasets}_r2']):

        for i in tqdm(range(10)): 
            result_for_iteration = dict()

            for method in tqdm(['exploration', 'exploitation', 'random']):   
                result_for_iteration[method] = active_learning_regression(dataset,method)
            result[dataset][i] = result_for_iteration
        with open(f'results_for_{dataset}_{best_or_worst_datasets}_r2.pickle', 'wb') as f:
            pickle.dump(result, f)
    return result